In [ ]:
# 필요한 패키지 설치
%pip install -qU langchain langchain-openai langchain-community langchain-text-splitters pypdf python-dotenv langsmith

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키가 설정되었는지 확인
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API 키가 설정되었습니다.")
else:
    print("❌ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인해주세요.")


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 경로 설정 (OpenAI RAG 실습과 동일한 문서 사용)
file_path = "docs/DeepSeek_OCR_paper.pdf"

# PDF 로더 초기화 및 문서 로드
loader = PyPDFLoader(file_path)
docs = loader.load()

print(f"📄 로드된 문서 수: {len(docs)} 페이지")


In [ ]:
# 첫 번째 페이지 내용 확인
print(f"📝 첫 번째 페이지 내용 (처음 500자):\n")
print(docs[0].page_content[:500])
print("\n" + "="*50)
print(f"\n📋 메타데이터: {docs[0].metadata}")


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
# - chunk_size: 각 청크의 최대 문자 수
# - chunk_overlap: 청크 간 겹치는 문자 수 (문맥 유지를 위해)
# - add_start_index: 원본 문서에서의 시작 위치 추적

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # 청크 크기 (문자 수)
    chunk_overlap=200,    # 청크 간 겹침 (문자 수)
    add_start_index=True  # 원본 문서에서의 인덱스 추적
)

# 문서 분할
all_splits = text_splitter.split_documents(docs)

print(f"✂️ 원본 문서 {len(docs)}페이지가 {len(all_splits)}개의 청크로 분할되었습니다.")


In [ ]:
# 분할된 청크 샘플 확인
print("📦 첫 번째 청크:")
print(f"내용: {all_splits[0].page_content[:300]}...")
print(f"\n메타데이터: {all_splits[0].metadata}")
print(f"\n청크 길이: {len(all_splits[0].page_content)} 문자")


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 인메모리 벡터 저장소 생성 및 문서 인덱싱
vector_store = InMemoryVectorStore.from_documents(
    documents=all_splits,
    embedding=embeddings
)

print("✅ 벡터 저장소 생성 완료!")


In [ ]:
# 유사도 검색 테스트
query = "DeepSeek-OCR 모델의 주요 특징은 무엇인가요?"

# 상위 3개의 유사한 문서 검색
similar_docs = vector_store.similarity_search(query, k=3)

print(f"🔍 검색 쿼리: {query}\n")
for i, doc in enumerate(similar_docs, 1):
    print(f"[결과 {i}]")
    print(f"📄 페이지: {doc.metadata.get('page', 'N/A')}")
    print(f"📝 내용: {doc.page_content[:200]}...\n")


In [ ]:
from langchain_openai import ChatOpenAI

# 채팅 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Retriever 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

print("✅ 2-Step RAG 구성 요소 준비 완료!")


In [ ]:
def rag_2step(question: str) -> dict:
    """2-Step RAG 구현: 검색 후 생성"""
    
    # Step 1: 관련 문서 검색
    retrieved_docs = retriever.invoke(question)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    
    # Step 2: LLM으로 답변 생성
    system_prompt = f"""You are a helpful assistant who is good at analyzing source information and answering questions.
Use the following source documents to answer the user's questions.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Answer in Korean.

Documents:
{docs_content}"""
    
    response = llm.invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ])
    
    return {
        "question": question,
        "answer": response.content,
        "source_documents": retrieved_docs
    }


In [ ]:
# 2-Step RAG 테스트
question = "DeepSeek-OCR 모델의 주요 특징은 무엇인가요?"
result = rag_2step(question)

print(f"❓ 질문: {result['question']}\n")
print(f"💡 답변: {result['answer']}\n")
print(f"📚 참조 문서 수: {len(result['source_documents'])}개")


In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent
from langsmith import traceable

# @tool 데코레이터를 사용하여 검색 도구 정의
# response_format="content_and_artifact"로 설정하면 문서 메타데이터도 함께 반환됩니다

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """DeepSeek-OCR 논문에서 관련 정보를 검색합니다.
    
    Args:
        query: 검색할 내용을 설명하는 쿼리 문자열
    
    Returns:
        검색된 문서의 내용과 메타데이터
    """
    retrieved_docs = retriever.invoke(query)
    
    # 문서 내용을 문자열로 결합
    content = "\n\n---\n\n".join(
        f"[페이지 {doc.metadata.get('page', 'N/A')}]\n{doc.page_content}" 
        for doc in retrieved_docs
    )
    
    # artifact에 원본 문서 메타데이터 저장
    artifact = [
        {"page": doc.metadata.get('page'), "source": doc.metadata.get('source')}
        for doc in retrieved_docs
    ]
    
    return content, artifact

print("✅ 검색 도구가 정의되었습니다.")


In [ ]:
# RAG 에이전트 생성
tools = [retrieve_context]

system_prompt = """You are a helpful assistant specialized in analyzing academic papers.
You have access to a tool that retrieves context from the DeepSeek-OCR paper.
Use the tool whenever you need specific information from the paper.

Guidelines:
- Always use the retrieve_context tool to find relevant information before answering
- If the first search doesn't provide enough information, try different search queries
- Answer in Korean
"""

rag_agent = create_agent(
    model="gpt-4.1-mini",
    tools=tools,
    system_prompt=system_prompt
)

print("✅ RAG 에이전트가 생성되었습니다.")


In [ ]:
# RAG 에이전트 테스트 - 단일 검색이 필요한 질문
question = "DeepSeek-OCR의 압축률(compression ratio)은 얼마인가요?"

print(f"❓ 질문: {question}\n")
print("🤖 에이전트 응답 과정:\n")

for event in rag_agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()


In [ ]:
# RAG 에이전트 테스트 - 복합 검색이 필요한 질문
complex_question = """DeepSeek-OCR의 아키텍처를 설명해주세요.
그리고 OmniDocBench에서의 성능은 어떤가요?"""

print(f"❓ 복합 질문: {complex_question}\n")
print("🤖 에이전트 응답 과정:\n")

for event in rag_agent.stream(
    {"messages": [{"role": "user", "content": complex_question}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()
